In [3]:
import pandas as pd
import os
import re
from bs4 import BeautifulSoup
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.initializers import Constant
from nltk.tokenize import  TweetTokenizer
import gensim
import csv
from sacremoses import  MosesDetokenizer

slang_dict = {}
listLabel = []
listWordBully = []
textPost = []

df = pd.read_csv("data/formspring_data.csv", sep='\t')
df.head()

,userid,post,ques,ans,asker,ans1,severity1,bully1,ans2,severity2,bully2,ans3,severity3,bully3
0,aguitarplayer94,Q: what&#039;s your favorite song? :D<br>A: I ...,what&#039;s your favorite song? :D<br>,I like too many songs to have a favorite,None,No,0,NaN,No,0,NaN,No,0,NaN
1,aprilpooh15,Q: <3<br>A: </3 ? haha jk! <33,<3,</3 ? haha jk! <33,None,No,0,NaN,No,0,NaN,No,0,NaN
2,aprilpooh15,Q: &quot;hey angel you duh sexy&quot;<br>A: R...,&quot;hey angel you duh sexy&quot;,Really?!?! Thanks?! haha,None,No,0,NaN,No,0,NaN,No,0,NaN
3,aprilpooh15,Q: (:<br>A: ;(,(:,;(,None,No,0,NaN,No,0,NaN,No,0,NaN
4,aprilpooh15,Q: ******************MEOWWW*******************...,******************MEOWWW*************************,*RAWR*?,None,No,0,NaN,No,0,NaN,No,0,NaN


In [5]:
def to_lowercase(text):
    if(text != None and text != ""):
        text = text.lower()
    return text

def strip_html(text):
    text_new=""
    if(text != None and text != ""):
        soup = BeautifulSoup(text, "html.parser")
        text_new = soup.get_text()
    return text_new

def remove_url(text):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)

def remove_punctuation(text):
    text = re.sub(r"[^\w\d'\s]+", '', text)
    return text

def convert_unicode_039(text):
    text = re.sub(r'&#039;',"'",text)
    return text

def convert_unicode_u2019(text):
    text = re.sub(r'u2019',"'",text)
    return text

def openSlang():
    with open('data/slang_dict.txt', 'r', encoding='ISO-8859-1') as slang:
        for line in csv.reader(slang, delimiter='\t'):
            slang_dict[line[-2]] = line[-1]


def preprocess_text(text):

    text = gensim.utils.any2unicode(text)

    text = convert_unicode_039(text)

    text = convert_unicode_u2019(text)

    text = remove_url(text)

    text = strip_html(text)

    text = remove_punctuation(text)

    tknzr = TweetTokenizer()
    text = tknzr.tokenize(text)

    output = [to_lowercase(word) for word in text if not word.isdigit()]
    ww = []

    for token in output:
        if token in slang_dict.keys():
            ww.append(token.replace(token, slang_dict[token]))
        else:
            ww.append(token)

    return ww

#sample output
samplePost = df['post']
for text in samplePost:
    text = str(text).replace("Q:", "")
    text = str(text).replace("A:", "")
    print(preprocess_text(text=str(text)))

["what's", 'your', 'favorite', 'song', 'd', 'i', 'like', 'too', 'many', 'songs', 'to', 'have', 'a', 'favorite']
['haha', 'jk']
['hey', 'angel', 'you', 'duh', 'sexy', 'really', 'thanks', 'haha']
[]
['meowww', 'rawr']
['any', 'makeup', 'tips', 'i', 'suck', 'at', 'doing', 'my', 'makeup', 'lol', 'sure', 'like', 'tell', 'me', 'wht', 'u', 'wnna', 'know', 'like', 'wht', 'do', 'you', 'use']
['apriiiiiiiiiiiill', 'i', 'miss', 'uuuu', "it's", 'emma', 'btw', 'haha', 'emma', 'hahahahah', 'd', 'i', 'missssseddd', 'yuhhhhh', '222222 3333']
['are', 'you', 'a', 'morning', 'or', 'night', 'person', 'night', '4shuree']
['are', 'you', 'a', 'trusting', 'person', 'alreadi', 'answrd']
['are', 'you', 'a', 'trusting', 'person', 'yes', 'veryy', 'trustin', 'person', 'may', 'i', 'help', 'yuh']
['are', 'you', 'and', 'truace', 'best', 'friends', 'ahaha', 'yess', 'of', 'course', 'aint', 'dat', 'rite', 'truace', 'haha', 'he', 'so', 'awesome', 'tho', 'bck', 'off', 'lol', 'jk']
['are', 'you', 'more', 'of', 'a', 'talker

['you', 'should', 'ask', 'her', 'if', 'shes', 'still', 'feelin', 'it', 'so', 'you', 'know', 'for', 'sure', 'and', 'why', 'are', 'you', 'trying', 'to', 'give', 'me', 'advice']
['you', 'tell', 'me', 'p', 'i', 'would', 'if', 'i', 'could', 'but', 'i', 'cant']
['youd', 'fight', 'for', 'lindsey', 'im', 'a', 'gurl', 'id', 'love', 'to', 'see', 'it', 'hahaha', 'well', 'then', 'i', "can't", 'fight', 'you', 'cani', 'and', 'if', 'you', "don't", 'mind', 'me', 'asking', 'why', 'do', 'you', 'dislike', 'her']
['your', 'a', 'fagggggg', "you're", 'a', 'bitchhhhh']
['your', 'mighty', 'attractive', 'd', 'aww', 'thanks', 'i', 'wonder', 'who', 'this', 'could', 'be']
['your', 'my', 'bitch', 'and', 'ill', 'kill', 'you', 'nigga', 'how', 'bout', 'you', 'suck', 'my', 'dick']
['your', 'so', 'gay', 'haha', 'well', "you're", 'fat']
['your', 'still', 'gay', 'and', 'a', 'bitch', 'haha', 'well', "you're", 'a', 'rapist', 'so', 'fuckoff']
['your', 'the', 'one', 'talkin', 'shit', 'back', 'fuckstick', 'i', 'mean', 'seriou

['ever', 'felt', 'that', 'no', 'one', 'relates', 'to', 'you', 'yes', 'all', 'the', 'time']
['ever', 'gave', 'a', 'really', 'long', 'apology', 'yea']
['ever', 'looked', 'at', 'a', 'guy', 'and', 'then', 'checked', 'out', 'his', 'package', 'yes']
['ever', 'seen', 'a', 'therapist', 'nope', 'but', 'sometimes', 'i', 'sure', 'wish', 'i', 'had', 'someone', 'i', 'could', 'pour', 'all', 'my', 'thoughts', 'out', 'too']
['ever', 'tought', 'about', 'going', 'to', 'the', 'moon', 'or', 'anouther', 'planet', 'yes', 'all', 'the', 'time']
['ever', 'tried', 'an', 'ouija', 'board', 'yup', 'have', 'one']
['ever', 'tried', 'dancing', 'in', 'the', 'rain', 'yes', 'slow', 'danced', 'with', 'my', 'boyfriend']
['ever', 'tried', 'skinny', 'dipping', 'with', 'whom', 'yes', 'with', 'my', 'sister', 'haha', 'we', 'went', 'night', 'swimming', 'in', 'our', 'pool', 'in', 'our', 'backyard', 'just', 'so', 'we', 'could', 'answer', 'this', 'question']
['ever', 'won', 'a', 'spelling', 'bee', 'nope']
['ever', 'wondered', 'wha

['omgee', 'i', 'get', 'super', 'shy', 'when', 'i', 'get', 'a', 'brazilian', 'wax', 'do', 'you', 'girls', 'get', 'shy', 'too', 'if', 'you', 'get', 'one', 'or', 'your', 'like', 'we', 'guys', 'do', 'you', 'like', 'your', 'girls', 'waxed', 'down', 'there', 'or', 'it', 'dont', 'matter', 'nope', 'i', 'dont', 'do', 'that', 'lol', 'too', 'expensive', 'i', 'have', 'other', 'thing', 'i', 'could', 'put', 'that', 'money', 'toward']
['on', 'a', 'scale', 'of', 'how', 'would', 'you', 'rate', 'yourself', 'personality', '9r', 'physic', '8r', 'looks']
['on', 'average', 'how', 'many', 'texts', 'do', 'you', 'send', 'a', 'day', 'lol']
['one', 'fun', 'thing', 'you', 'did', 'one', 'ive', 'done', 'lots', 'of', 'fun', 'things', 'i', 'go', 'to', 'parks', 'and', 'rollerblade', 'ive', 'been', 'to', 'panama', 'beach', 'with', 'other', 'girls', 'for', 'vaca', 'i', 'got', 'to', 'spend', 'an', 'entire', 'weekend', 'with', 'my', 'family', 'this', 'past', 'weekend', 'i', 'paint', 'colorful', 'paintingslots', 'of', 'fun

['why', 'do', 'you', 'think', 'people', 'join', 'gangs', 'i', 'have', 'no', 'idea', 'nothing', 'else', 'to', 'do', 'in', 'life']
['why', 'do', 'you', 'think', 'people', 'lie', 'because', 'they', 'have', 'nothing', 'better', 'to', 'say']
['why', 'do', 'you', 'think', 'people', 'stalk', 'other', 'people', 'yes']
['why', 'does', 'a', 'round', 'pizza', 'come', 'in', 'a', 'square', 'box', "i've", 'soooo', 'wondered', 'that', 'too']
['why', 'doesnt', 'tarzan', 'have', 'a', 'beard', 'he', 'shaves']
['why', 'doesnt', 'tarzan', 'have', 'a', 'beard', 'idk', 'good', 'question']
['why', 'doesnt', 'winnie', 'the', 'pooh', 'ever', 'get', 'stung', 'by', 'the', 'bees', 'he', 'messes', 'with', 'causes', 'they', "can't", 'have', 'violence', 'in', 'the', 'kids', 'show', 'come', 'on', 'you', 'knew', 'thathahah']
['why', "don't", 'u', 'answer', 'ur', 'phone', 'idk', 'dude', 'why', 'do', 'you', 'call', 'so', 'much', 'haha']
['why', 'doy', 'ou', 'think', 'we', 'are', 'greedy', 'is', 'this', 'by', 'nature', '

...

['what', 'is', 'your', 'style', 'my', 'style', 'you', 'mean', 'with', 'hair', 'makeup', 'clothing', 'etc', 'shall', 'i', 'answer', 'all', 'lol', 'so', 'hair', 'style', "i'm", 'not', 'one', 'of', 'those', 'people', 'who', 'like', 'their', 'hair', 'to', 'look', 'perfect', 'all', 'the', 'time', 'on', 'occasion', 'yeah', 'i', 'like', 'to', 'style', 'it', 'but', 'im', 'not', 'anal', 'about', 'it', 'being', 'perfect', 'i', 'absolutely', 'love', 'messy', 'hair', 'so', 'i', 'use', 'the', 'natural', 'wave', 'thats', 'in', 'my', 'hair', 'to', 'create', 'a', 'messy', 'and', 'kind', 'of', 'bedhead', 'look', 'i', 'use', 'more', 'hairspray', 'than', 'i', 'probably', 'should', 'but', 'i', 'like', 'the', 'end', 'effect', 'my', 'makeup', 'style', 'i', 'prefer', 'a', 'natural', 'makeup', 'look', 'for', 'daytime', 'for', 'example', 'a', 'little', 'eyeliner', 'and', 'gold', 'tone', 'eye', 'shadows', 'for', 'evening', 'or', 'clubs', 'depending', 'on', 'my', 'clothing', 'i', 'love', 'a', 'black', 'smokey', 

['idgaf', 'lame', 'bitch', 'lol', 'whyy', 'are', 'you', 'still', 'on', 'my', 'dickk', 'wtffff', 'do', 'you', 'wantt']
['if', 'you', 'want', 'me', 'to', 'spam', 'you', 'ask', 'nah', 'thanks', 'd']
['lady', 'gaga', 'xtina', 'me', 'encantan', 'no', 'm', 'imxta', 'nd', 'd', 'lo', 'q', 'digan', 'd', 'las', 'a', 'mi', 'me', 'gustaba', 'cristina', 'pero', 'ps', 'ya', 'esta', 'toda', 'odiosa', 'xd']
['u2665', 'u2665', 'u2665']
['_', 'emor', 'lmao', 'ahaha', 'love', 'it', 'p']
['hehe', "that's", 'funny', 'i', 'want', 'to', 'fuck', 'your', 'tight', 'ass', 'o', 'haha', 'maybe', 'i', 'know', 'who', 'you', 'are', 'd']
['t', 'intereso', 'hahaha', 'broma', 'r', 'n', 'serio', 'muchas', 's', 'r', 'uu', 'no', 'seras', 'd', 'esos', 'chicos', 'u00bfno', 'xd', 'interesar', 'es', 'muy', 'diferente', 'a', 'ir', 'a', 'tener', 'algo']
['kiss', 'back', 'lol', 'u2665u2665']
['kisses', 'muah', 'd', 'ahaha', 'p', 'wanna', 'know', 'more', 'just', 'ask']
["'", 'd']
['what', 'country', 'are', 'you', 'in', 'colombia',

['do', 'you', 'have', 'any', 'of', 'your', 'childhood', 'stuffed', 'animals', 'which', 'one', 'what', 'iswas', "it's", 'name', 'my', 'first', 'teddy', 'bear', 'pooky', 'is', 'now', 'years', 'old']
['do', 'you', 'have', 'any', 'petsr', 'cat', 'and', 'dog']
['do', 'you', 'have', 'any', 'phobias', 'what', 'are', 'they', "i'm", 'frightened', 'of', 'porn', 'baha', 'xd', 'and', 'ants', 'haha', 'death']
['do', 'you', 'have', 'any', 'weird', 'habits', 'i', 'have', 'several', 'i', 'am', 'not', 'going', 'into', 'them', 'right', 'now']
['do', 'you', 'have', 'any', 'younger', 'friends', 'or', 'even', 'family', 'that', 'are', 'already', 'grazing', 'your', 'height', 'or', 'even', 'taller', 'than', 'you', 'my', 'younger', 'brother', 'years', 'younger', 'and', 'already', 'my', 'height', 'o', 'i', "can't", 'beat', 'him', 'up', 'anymore', 'lol', 'jk', 'my', 'youngest', 'nephew', 'is', 'only', 'and', 'already', 'ft', 'my', 'height']
['do', 'you', 'have', 'any', 'younger', 'siblings', 'if', 'so', 'age', '

['if', 'a', 'bfgfcrush', 'really', 'liked', 'an', 'outfit', 'on', 'you', 'that', 'you', 'found', 'to', 'be', 'unappealing', 'and', 'possibly', 'even', 'downright', 'ugly', 'would', 'you', 'wear', 'it', 'for', 'them', 'anyway', 'why', 'or', 'why', 'not', 'i', 'would', 'real', 'love', 'means', 'being', 'to', 'do', 'things', 'that', 'you', "wouldn't", 'do', 'by', 'your', 'self']
['if', 'a', 'cop', 'pulled', 'you', 'over', 'and', 'asked', 'to', 'go', 'through', 'your', 'vehicle', 'and', 'you', "don't", 'have', 'anything', 'to', 'be', 'ashamed', 'of', 'would', 'you', 'let', 'them', 'why', 'no', 'it', 'violets', 'my', '4th', 'ammendment', 'right']
['if', 'a', 'new', 'medicine', 'were', 'developed', 'that', 'would', 'cure', 'alzhiemers', 'but', 'cause', 'a', 'fatal', 'reaction', 'in', 'of', 'those', 'who', 'took', 'it', 'would', 'you', 'want', 'it', 'to', 'be', 'released', 'to', 'the', 'public', 'yes', "that's", 'acceptable', 'losses']
['if', 'a', 'random', 'stranger', 'approached', 'you', 's

['what', 'is', 'one', 'article', 'of', 'clothing', 'or', 'clothing', 'style', 'you', 'absolutely', 'cannot', 'stand', 'on', 'the', 'opposite', 'sex', 'for', 'instance', 'i', 'hate', 'my', 'men', 'in', 'flip', 'flops', 'women', 'in', 'ball', 'caps']
['what', 'is', 'one', 'article', 'of', 'clothing', 'you', 'wish', 'would', 'die', 'off', 'tie', 'diy', 'tee', 'shirts']
['what', 'is', 'one', 'of', 'your', 'talents', 'that', 'not', 'many', 'people', 'know', 'of', 'i', 'can', 'draw']
['what', 'is', 'one', 'personality', 'trait', 'that', 'you', 'are', 'ashamed', 'of', 'for', 'having', 'being', 'jelous']
['what', 'is', 'one', 'thing', 'that', 'should', 'only', 'be', 'acceptable', 'for', 'the', 'opposite', 'gender', 'to', 'do', 'wear', 'dresses']
['what', 'is', 'one', 'thing', 'that', 'you', 'absolutely', 'cannot', 'stand', 'and', 'why', 'is', 'this', 'can', 'we', 'prevent', 'this', 'as', 'a', 'society', 'if', 'so', 'how', 'sloppy', 'people']
['what', 'is', 'one', 'thing', 'that', 'you', 'did',

In [11]:
slang_dict = {}
listLabel = []
listWordBully = []
textPost = []

df = pd.read_csv("data/formspring_data.csv", sep='\t')
df.head()

def loadData():
    openSlang()

    listPost = df['post']
    md = MosesDetokenizer()
    for text in listPost:
        text = str(text).replace("Q:", "")
        text = str(text).replace("A:", "")
        textPost.append(md.detokenize(preprocess_text(text=str(text))))
    df['post'] = textPost

    countYes = 0
    for i in range(len(df)):

        ans1 = str(df["ans1"][i])
        ans2 = str(df["ans2"][i])
        ans3 = str(df["ans3"][i])

        bull1 = str(df["bully1"][i])
        bull2 = str(df["bully2"][i])
        bull3 = str(df["bully3"][i])

        if ans1.lower() == "yes" or ans2.lower() == "yes" or ans3.lower() == "yes":
            countYes = countYes + 1
            if countYes > 1:
                listLabel.append("yes")
                countYes = 0
            else:
                listLabel.append("no")
        else:
            listLabel.append("no")

        if bull1.lower() != "n/a" and bull1.lower() != "none" and bull1.lower() != "nan":
            listWordBully.append(bull1)
        if bull2.lower() != "n/a" and bull2.lower() != "none" and bull2.lower() != "nan":
            listWordBully.append(bull2)
        if bull3.lower() != "n/a" and bull3.lower() != "none" and bull3.lower() != "nan":
            listWordBully.append(bull3)
loadData()  
#sample output slang dictionary
print(slang_dict)

{'': '', '0kkay': 'okay', '0n': 'on', '0neee': 'one', '0r': 'or', '1s': 'once', '2daii': 'today', '2day': 'today', "today's": 'today', '2gether': 'together', '2marro': 'tomorrow', '2moro': 'tomorrow', '2morro': 'tomorrow', '2morrow': 'tomorrow', '2moz': 'tomorrow', '2mz': 'tomorrow', '2nd': 'second', '2niiqht': 'tonight', '2nite': 'tonight', '2nyt': 'tonight', 'two': 'tomorrow', '3s': 'threes', '4evaa': 'forever', '4ever': 'forever', '4got': 'forgot', '4rm': 'from', '5ay': 'say', '5ayin': 'saying', '5o': 'so', '5th': 'fifth', '5top': 'stop', 'aa': 'a', 'aaaand': 'and', 'aaaaw': 'anyway', 'aaalll': 'all', 'aand': 'and', 'abole': 'able', 'abooout': 'about', 'absolotuely': 'absolutely', 'absolutly': 'absolutely', 'abt': 'about', 'accross': 'across', 'acct': 'account', 'achivements': 'achievements', 'acount': 'account', 'actin': 'acting', 'actn': 'action', 'actt': 'act', 'actualy': 'actually', 'addaction': 'addiction', 'addin': 'adding', 'addiquate': 'adequate', 'addres': 'address', 'addy'

In [12]:
max_features = 20000

tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', filters=' ')
tokenizer.fit_on_texts(df['post'].values)

X = tokenizer.texts_to_sequences(df['post'].values)

X = pad_sequences(X, 50)

y = pd.get_dummies(df['bully1']).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print("test set size " + str(len(X_test)))

test set size 1278


In [13]:
embeddings_index = {}
f = open('data/glove.6B.300d.txt',encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [14]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 19273 unique tokens.


In [15]:
num_words = min(max_features, len(word_index)) + 1
print(num_words)

embedding_dim = 300

embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.random.randn(embedding_dim)

19274


In [18]:
model = Sequential()
model.add(Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=50,
                    trainable=True))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.25))
model.add(Dense(units=2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

batch_size = 128
history = model.fit(X_train, y_train, epochs=5, batch_size=batch_size, verbose=1, validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 300)           5782200   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 50, 300)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 50, 128)           186880    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 64)                41216     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 6,010,426
Trainable params: 6,010,426
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking target: expected dense_2 to have shape (2,) but got array with shape (1013,)

ValueError: Error when checking target: expected dense_1 to have shape (2,) but got array with shape (1013,)

In [19]:
y_hat = model.predict(X_test)
print(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat))))

0.03599374021909233
